In [ ]:
import json
import os

outdir = './output'
if not os.path.exists(outdir):
    os.mkdir(outdir)
with open('bus_routes.json', 'r') as f:
  data = json.load(f)

In [ ]:
service = "858"
direction = "1"
route = data[service][direction]

In [ ]:
od = []
for seq, id in enumerate(route[:-1]):
    next_stop = seq + 1
    for stop in route[next_stop:]:
        if id['code'] == stop['code']:
            break #we've come back to the same bus stop. assume all pax from here on out boarded on 2nd visit.
        od.append((int(id['code']), int(stop['code'])))

In [ ]:
import pandas as pd
#to use:
#copy origin_destination_summary from the data folders into the working directory, rename to "od.csv" or change filename below
df_source = pd.read_csv("od.csv")

In [ ]:
df = pd.DataFrame()
for src, dest in od:
    df = pd.concat([df, df_source[(df_source['ORIGIN_PT_CODE'] == src) & (df_source['DESTINATION_PT_CODE'] == dest)]])

#retrieve the filtered report for the given service and direction
df.to_csv(f"output/{service}_{direction}.csv")

In [ ]:
import math

# we keep only the first digit of the stop code as it generally conforms to geographical areas.
# second digit may also be useful in some cases but could create too much statistical noise.
# for example, both CCK and BP have a mix of stop codes in both 44000 and 45000 ranges
def rounder(x):
    return int(math.floor(x/10000)) * 10000

df_round = df
df_round.ORIGIN_PT_CODE = df_round.ORIGIN_PT_CODE.apply(rounder)
df_round.DESTINATION_PT_CODE = df_round.DESTINATION_PT_CODE.apply(rounder)
df_round.groupby(["DAY_TYPE", "ORIGIN_PT_CODE", "DESTINATION_PT_CODE"]).sum()